# Importar Lib's

In [1]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from PyPDF2 import PdfReader
from IPython.display import display
from datetime import datetime
from ipywidgets import interact
from openpyxl import Workbook

# Passo 1: Listar arquivos PDF em uma pasta específica

In [2]:
diretorio_bf = r"C:\Users\OneEngInst\Projetos_python\dashboard\content\base_sip_Concluido.xlsx"

In [3]:
# Diretório onde estão os PDFs
directory = r"C:\Users\OneEngInst\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\001_INTELBRAS\AC\1_0-CORRETIVA\1ªTRANCHE\2024"
bdsip = pd.read_excel(diretorio_bf)

# Lista para armazenar os dados
data = []

# Função para extrair o valor após um termo específico
def extract_value(text, term):
    if term in text:
        start_index = text.find(term) + len(term)
        # Aqui consideramos que o valor pode estar logo abaixo do termo, em uma nova linha
        value = text[start_index:].splitlines()[1].strip()
        return value
    return None

# Função para processar um arquivo PDF
def process_pdf(filepath):
    filename = os.path.basename(filepath)
    reader = PdfReader(filepath)
    print(f"Lendo arquivo: {filename}")
    
    # Dicionário para armazenar os valores extraídos de cada PDF
    pdf_data = {"Arquivo": filename}
    
    # Percorrer todas as páginas do PDF
    for page in reader.pages:
        text = page.extract_text()
        
        # Extrair os valores desejados
        sigfi = extract_value(text, "SIGFI:")
        num_os = extract_value(text, "NUM OS:")
        
        # Armazenar os valores no dicionário
        if sigfi:
            pdf_data["SIGFI"] = sigfi
        if num_os:
            pdf_data["NUM OS"] = num_os
    
    # Adicionar o dicionário ao conjunto de dados, se tiver valores relevantes
    if pdf_data.get("SIGFI") or pdf_data.get("NUM OS"):
        data.append(pdf_data)

# Percorrer todos os arquivos no diretório e subdiretórios
for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".pdf"):
            filepath = os.path.join(root, filename)
            process_pdf(filepath)

Lendo arquivo: 641703_100633_CARLOS_COSTA_DOS_SANTOS_14604.pdf
Lendo arquivo: 642332_115323_CLAUDECY_DOS_SANTOS_DE_MENEZES_13529.pdf
Lendo arquivo: 642484_101410_JOSE_MARIA_NUNES_DA_COSTA_13536.pdf
Lendo arquivo: 642558_103708_MARIA_JARLENE_TAVARES_DA_COSTA_13746.pdf
Lendo arquivo: 642567_103693_MARIANA_GOMES_DO_NASCIMENTO_13404.pdf
Lendo arquivo: 642753_104106_MARIA_DAISIA_LIMA_DE_SOUZA_14606.pdf
Lendo arquivo: 642764_105481_VANUZIA_SILVA_DE_CARVALHO_13730.pdf
Lendo arquivo: 642764_116297_VANUZIA_SILVA_DE_CARVALHO_13730.pdf
Lendo arquivo: 642769_109323_FRANCISCO_COSTA_TAVARES_13750.pdf
Lendo arquivo: 642864_99855_MARIA_LUZIA_DUTRA_DA_SILVA_13752.pdf
Lendo arquivo: 642939_102510_FRANCISCO_GELIDO_FIRMINO_MIRANDA_13498.pdf
Lendo arquivo: 642941_101482_JOSE_GOMES_FIRMINO_13483.pdf
Lendo arquivo: 642946_103703_JOSE_FRANCISCO_DA_COSTA_13524.pdf
Lendo arquivo: 642962_112156_MARIA_ELIZABETE_PEREIRA_DOS_SANTOS_14234.pdf
Lendo arquivo: 643057_105002_MARIA_DAS_DORES_MARQUES_DA_SILVA_13740.pdf
Le

In [4]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\OneEngInst\Projetos_python\dashboard\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 9, 17), step=1)

## Pasta contendo os arquivos PDF

In [7]:
datareader = pd.DataFrame(data)
datareader["NUM OS"] = datareader["NUM OS"].astype('int64')
dataset = datareader.rename(columns={"NUM OS":"NUMOS"})
dataset = dataset.merge(bdsip, left_on='NUMOS', right_on='NUMOS', how='left')
numocorrence = dataset[['NUMOCORRENCIA']]

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 57 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Arquivo            825 non-null    object        
 1   SIGFI              825 non-null    object        
 2   NUMOS              825 non-null    int64         
 3   Unnamed: 0         825 non-null    bool          
 4   ORIGEM             825 non-null    object        
 5   TIPO               825 non-null    object        
 6   NUMOCORRENCIA      821 non-null    object        
 7   REINCID            825 non-null    int64         
 8   ROTA               825 non-null    float64       
 9   NOMECLIENTE        825 non-null    object        
 10  CPF                825 non-null    object        
 11  CODIGOENERGISA     825 non-null    object        
 12  UC                 824 non-null    float64       
 13  IDSIGFI            825 non-null    int64         
 14  QDCACCNS  

In [9]:
def encontrar_duplicados(numocorrencia):
    duplicados = numocorrencia[numocorrencia.duplicated(subset='NUMOCORRENCIA')]
    return duplicados

# arquivos_pdf = []
# for raiz, diretorios, arquivos in os.walk(diretorio_lote):
#     for arquivo in arquivos:
#         if arquivo.endswith('.pdf'):
#             arquivos_pdf.append(os.path.join(raiz, arquivo))

# string = pd.DataFrame(pd.Series(arquivos_pdf).str.extract(r"(\d+)_(\d+)_(.+)_(.+)_(.+)_(.+)"))
# string.rename(columns={1: 'NUMOCORRENCIA'}, inplace=True)
#numocorrence = numocorrence[['NUMOS']].astype('float')  # Convertendo para float para acomodar NaN

# Contagem total de duplicatas
duplicadas_count = numocorrence[numocorrence.duplicated(subset='NUMOCORRENCIA')].shape[0]

# Encontrar duplicatas
date_duplicated= encontrar_duplicados(numocorrence.copy())  # Cria uma cópia independente
date_duplicated.dropna(inplace=True)  # Remove NaNs do DataFrame duplicatas
date_duplicated['NUMOCORRENCIA'] = date_duplicated['NUMOCORRENCIA'].astype('int')  # Convertendo de volta para int
# Mesclando os dados principais.
df_pdf = numocorrence.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
df_pdf = df_pdf[['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]
print("Número total de duplicatas:", duplicadas_count)
print("Duplicatas encontradas após remoção de NaN:")
display(df_pdf)

Número total de duplicatas: 18
Duplicatas encontradas após remoção de NaN:


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
0,2.024001e+11,641703.0,14604.0,100633,CARLOS COSTA DOS SANTOS,1.0,VALIDADO,ONEOM003,2024-08-02 10:41:00
1,2.024001e+11,642332.0,13529.0,115323,CLAUDECY DOS SANTOS DE MENEZES,6.0,VALIDADO,ONEOM003,2024-09-05 11:35:00
2,2.024001e+11,642484.0,13536.0,101410,JOSE MARIA NUNES DA COSTA,6.0,VALIDADO,ONEOM024,2024-08-01 12:12:00
3,2.024001e+11,642558.0,13746.0,103708,MARIA JARLENE TAVARES DA COSTA,1.0,VALIDADO,ONEOM024,2024-08-07 14:18:00
4,2.024001e+11,642567.0,13404.0,103693,MARIANA GOMES DO NASCIMENTO,1.0,VALIDADO,ONEOM024,2024-08-07 08:41:00
...,...,...,...,...,...,...,...,...,...
906,2.024001e+11,680520.0,14135.0,95780,PEDRO FERREIRA DAS NEVES,3.0,VALIDADO,ONEOM018,2024-07-11 12:23:00
907,2.024001e+11,680614.0,13489.0,97560,MARIA DE FATIMA SANTANA DE OLIVEIRA,6.0,VALIDADO,ONEL004,2024-07-16 10:28:00
908,2.024001e+11,688897.0,15787.0,93617,JOSÉ MAURO DE AMORIM DOS SANTOS,13.0,VALIDADO,ONEL006,2024-07-12 10:44:00
909,2.024001e+11,699364.0,13738.0,95436,MARCIA ANTÔNIA LIMA DO NASCIMENTO,1.0,VALIDADO,ONEOM003,2024-07-12 10:21:00


# Lista de arquivos PDF na pasta

# Passo 2: Ler o arquivo Excel com a lista de arquivos

In [10]:
caminho_arquivo_excel = df_filtered_global_oem
caminho_arquivo_excel['NUMOCORRENCIA'].count()
df_excel = caminho_arquivo_excel [['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

In [11]:
df_excel

,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
66,202400069706,643063.0,13760,117551,FRANCISCA DANUBIA ROCHA DA COSTA,1.0,VALIDADO,ONEOM024,2024-09-07 14:15:00
74,202400069615,658170.0,13465,116590,FRANCISCO SILVA DE SOUZA,1.0,VALIDADO,ONEOM024,2024-09-08 14:20:00
75,202400069607,643061.0,14616,116546,ANTONIA ZENAIDE ALVES VIEIRA,1.0,VALIDADO,ONEOM024,2024-09-07 12:49:00
76,202400069599,642558.0,13746,116531,MARIA JARLENE TAVARES DA COSTA,1.0,VALIDADO,ONEOM024,2024-09-06 11:53:00
80,202400069540,645897.0,14621,116535,GEILSON SOUZA DA SILVA,1.0,VALIDADO,ONEOM024,2024-09-06 13:16:00
...,...,...,...,...,...,...,...,...,...
9785,202200024406,660291.0,14514,74058,MARIA SUELEN DA SILVA AGUIAR,7.0,VALIDADO,ONEOM004,2024-05-17 10:47:00
9804,202200021972,658224.0,13366,72643,MARIA SOCORRO OLIVEIRA DA SILVA,4.0,VALIDADO,ONEOM007,2024-05-27 16:36:00
9805,202200021881,658007.0,13348,69250,JOSE FRANCISCO LIMA DA SILVA,4.0,VALIDADO,ONEOM007,2024-05-27 15:31:00
9806,202200021758,656788.0,13297,71562,JOELSON CRUZ DA SILVA,4.0,VALIDADO,ONEOM007,2024-05-28 09:01:00


## DataFrame a partir do arquivo Excel


#  Passo 3: Comparar os DataFrames para identificar arquivos ausentes no PDF e arquivos não listados no Excel

In [12]:
arquivos_ausentes_no_pdf = df_excel[~df_excel['NUMOCORRENCIA'].isin(df_pdf['NUMOCORRENCIA'])]
arquivos_nao_listados_no_excel = df_pdf[~df_pdf['NUMOCORRENCIA'].isin(df_excel['NUMOCORRENCIA'])]

## Resultados

In [13]:
print(arquivos_ausentes_no_pdf['NUMOCORRENCIA'].count())
display(arquivos_ausentes_no_pdf)

37


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
76,202400069599,642558.0,13746,116531,MARIA JARLENE TAVARES DA COSTA,1.0,VALIDADO,ONEOM024,2024-09-06 11:53:00
99,202400068310,642567.0,13404,115750,MARIANA GOMES DO NASCIMENTO,1.0,VALIDADO,ONEOM024,2024-09-07 08:25:00
105,202400068245,657903.0,14017,114771,MARIA IRISANE SILVA DE SOUZA,3.0,VALIDADO,ONEOM007,2024-09-07 14:34:00
108,202400068211,652903.0,13454,114683,MARINALVA MACIEL FILHO,1.0,VALIDADO,ONEOM024,2024-09-06 17:10:00
110,202400068195,654702.0,14217,115315,VILCELIA SOUZA DE AMORIM,7.0,VALIDADO,ONEOM004,2024-09-07 12:47:00
111,202400068187,646358.0,14351,115162,ELISANIR DO CARMO DE SOUZA,7.0,VALIDADO,ONEOM012,2024-09-02 18:03:00
146,202400067007,748104.0,14028,113450,ÍTALO SOUZA BARROS,3.0,VALIDADO,ONEOM007,2024-08-27 17:37:00
156,202400066850,656687.0,13834,112646,FRANCISCO NASCIMENTO PINHEIRO,3.0,VALIDADO,ONEOM003,2024-08-27 17:29:00
329,202400060143,669424.0,15761,106174,RAIMUNDO PEREIRA DE LIMA,12.0,VALIDADO,ONEOM004,2024-08-21 11:51:00
1083,202400047835,656954.0,13550,90433,MARIA ERONIDES MARANGUAPE DE ARAUJO,9.0,VALIDADO,ONEOM012,2024-06-28 14:15:00


In [14]:
arquivos_nao_listados_no_excel['NUMOCORRENCIA'].count()
nao_listado_excel = arquivos_nao_listados_no_excel.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
nao_listado_excel

,NUMOS_x,UC_x,IDSIGFI_x,NUMOCORRENCIA,NOMECLIENTE_x,ROTA_x,STATUS_x,EXECUTOR_x,DTCONCLUSAO_x,Unnamed: 0,...,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG,IDEMPRESA,EMPRESA
0,NaN,NaN,NaN,160976,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,160957,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,131956,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,154833,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,166329,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,44465,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,137812,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
recorrence= df_filtered_global_oem.merge(date_duplicated, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')
recorrence['IDSIGFI'] = recorrence['IDSIGFI'].astype('int')

In [16]:
duplicated_numocorrencia = date_duplicated.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')

In [17]:
print("listar arquivo duplicados:")
print(recorrence.shape)
display(recorrence)

print("listar nº ocorrencia que esta duplicada")
display(sorted(duplicated_numocorrencia['NUMOCORRENCIA']))  

print("Arquivos ausentes na lista de PDF:")
print(arquivos_ausentes_no_pdf.shape)
display(arquivos_ausentes_no_pdf)

print("\nArquivos não listados no Excel:")
print(arquivos_nao_listados_no_excel.shape)
display(arquivos_nao_listados_no_excel)

listar arquivo duplicados:
(862, 55)


,Unnamed: 0,ORIGEM,TIPO,NUMOS,NUMOCORRENCIA,REINCID,ROTA,NOMECLIENTE,CPF,CODIGOENERGISA,...,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG,IDEMPRESA,EMPRESA
0,False,ENERGISA,CORRETIVA,202400069706,117551,1,1.0,FRANCISCA DANUBIA ROCHA DA COSTA,045.821.082-02,MLPA7217457-22,...,2024-09-07 13:44:00,2024-09-07 10:39:00,3,2024-09-07,202400069706,Report2,2666,NaN,1.0,INTELBRAS
1,False,ENERGISA,CORRETIVA,202400069615,116590,2,1.0,FRANCISCO SILVA DE SOUZA,021.942.502-71,MLPA6925765-21,...,2024-09-08 14:24:00,2024-09-05 00:00:00,3,2024-09-08,202400069615,Report2,2666,NaN,1.0,INTELBRAS
2,False,ENERGISA,CORRETIVA,202400069607,116546,5,1.0,ANTONIA ZENAIDE ALVES VIEIRA,872.641.452-04,MLPA7208068-22,...,2024-09-07 12:53:00,2024-09-05 00:00:00,3,2024-09-07,202400069607,Report2,2666,NaN,1.0,INTELBRAS
3,False,ENERGISA,CORRETIVA,202400069599,116531,1,1.0,MARIA JARLENE TAVARES DA COSTA,795.846.592-00,MLPA7217442-22,...,2024-09-06 11:55:00,2024-09-06 10:01:00,3,2024-09-06,202400069599,Report2,1500,NaN,1.0,INTELBRAS
4,False,ENERGISA,CORRETIVA,202400069540,116535,1,1.0,GEILSON SOUZA DA SILVA,000.528.482-13,MLPA7208073-22,...,2024-09-06 13:24:00,2024-09-05 13:51:00,3,2024-09-06,202400069540,Report2,2666,NaN,1.0,INTELBRAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,False,ENERGISA,CORRETIVA,202200024406,74058,0,7.0,MARIA SUELEN DA SILVA AGUIAR,770.368.672-72,MLPA7816611-22,...,2024-05-17 10:46:00,2024-05-10 14:25:00,3,2024-05-17,202200024406,Report2,2666,NaN,1.0,INTELBRAS
858,False,ENERGISA,CORRETIVA,202200021972,72643,1,4.0,MARIA SOCORRO OLIVEIRA DA SILVA,029.887.812-73,MLPA6705399-21,...,2024-05-27 16:34:00,2024-05-07 16:00:00,3,2024-05-27,202200021972,Report2,2666,NaN,1.0,INTELBRAS
859,False,ENERGISA,CORRETIVA,202200021881,69250,0,4.0,JOSE FRANCISCO LIMA DA SILVA,053.309.582-45,MLPA6726482-21,...,2024-05-27 15:30:00,2024-04-30 13:50:00,3,2024-05-27,202200021881,Report2,2666,NaN,1.0,INTELBRAS
860,False,ENERGISA,CORRETIVA,202200021758,71562,0,4.0,JOELSON CRUZ DA SILVA,064.628.662-50,MLPA6725915-21,...,2024-05-28 09:00:00,2024-05-05 13:00:00,3,2024-05-28,202200021758,Report2,2666,NaN,1.0,INTELBRAS


listar nº ocorrencia que esta duplicada


[67322,
 71974,
 72961,
 73625,
 74129,
 75901,
 76637,
 83542,
 86905,
 88557,
 90064,
 94820,
 94897,
 105988,
 105988,
 170525]

Arquivos ausentes na lista de PDF:
(37, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
76,202400069599,642558.0,13746,116531,MARIA JARLENE TAVARES DA COSTA,1.0,VALIDADO,ONEOM024,2024-09-06 11:53:00
99,202400068310,642567.0,13404,115750,MARIANA GOMES DO NASCIMENTO,1.0,VALIDADO,ONEOM024,2024-09-07 08:25:00
105,202400068245,657903.0,14017,114771,MARIA IRISANE SILVA DE SOUZA,3.0,VALIDADO,ONEOM007,2024-09-07 14:34:00
108,202400068211,652903.0,13454,114683,MARINALVA MACIEL FILHO,1.0,VALIDADO,ONEOM024,2024-09-06 17:10:00
110,202400068195,654702.0,14217,115315,VILCELIA SOUZA DE AMORIM,7.0,VALIDADO,ONEOM004,2024-09-07 12:47:00
111,202400068187,646358.0,14351,115162,ELISANIR DO CARMO DE SOUZA,7.0,VALIDADO,ONEOM012,2024-09-02 18:03:00
146,202400067007,748104.0,14028,113450,ÍTALO SOUZA BARROS,3.0,VALIDADO,ONEOM007,2024-08-27 17:37:00
156,202400066850,656687.0,13834,112646,FRANCISCO NASCIMENTO PINHEIRO,3.0,VALIDADO,ONEOM003,2024-08-27 17:29:00
329,202400060143,669424.0,15761,106174,RAIMUNDO PEREIRA DE LIMA,12.0,VALIDADO,ONEOM004,2024-08-21 11:51:00
1083,202400047835,656954.0,13550,90433,MARIA ERONIDES MARANGUAPE DE ARAUJO,9.0,VALIDADO,ONEOM012,2024-06-28 14:15:00



Arquivos não listados no Excel:
(7, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
371,NaN,NaN,NaN,160976,NaN,NaN,NaN,NaN,NaT
375,NaN,NaN,NaN,160957,NaN,NaN,NaN,NaN,NaT
384,NaN,NaN,NaN,131956,NaN,NaN,NaN,NaN,NaT
388,NaN,NaN,NaN,154833,NaN,NaN,NaN,NaN,NaT
402,NaN,NaN,NaN,166329,NaN,NaN,NaN,NaN,NaT
526,NaN,NaN,NaN,44465,NaN,NaN,NaN,NaN,NaT
530,NaN,NaN,NaN,137812,NaN,NaN,NaN,NaN,NaT


In [ ]:
#Salvando a planilha Manutenções Energisa_preventiva para ser enviadas em Excell( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\OneEngInst\Projetos_python\notebook\OEM\output\pdf_ausentes_corretivas_1_TRANCHE-geral.xlsx")
arquivos_ausentes_no_pdf.to_excel(dataframe_excel)
dataframe_excel.close()